<a href="https://colab.research.google.com/github/amylynnn/weatherwise-Amylynn-Sophie/blob/main/draftfinal.starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise



In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
import requests
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from datetime import datetime, timedelta



In [ ]:
API_KEY = '7cf20335110caaf78db0fecb31852d45'

# Simple in-memory cache: { (source, city): (timestamp, data) }
weather_cache = {}

CACHE_EXPIRY_MINUTES = 10  # Cache duration

## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
#import requests
#import matplotlib.pyplot as plt
#import seaborn as sns
#import plotly.express as px
#import pandas as pd
#import re

# I added two more codes to enhance the visualisation
#from fetch_my_weather import get_weather
#from hands_on_ai.chat import get_response
# Typing support for cleaner, testable function signatures
#from typing import Union, List, Dict

## 🌤️ Weather Data Functions

In [ ]:
# Define get_weather_data() function here
def fetch_weather_data(city, source='openweathermap'):
    """
    Fetch weather data from specified source with caching.
    """
    global weather_cache
    key = (source.lower(), city.lower())

    now = datetime.utcnow()
    if key in weather_cache:
        timestamp, cached_data = weather_cache[key]
        if now - timestamp < timedelta(minutes=CACHE_EXPIRY_MINUTES):
            # Use cached data
            return cached_data

    # No valid cache, fetch fresh data
    if source.lower() == 'openweathermap':
        data = fetch_openweathermap(city)
    elif source.lower() == 'mocksource':
        data = fetch_mock_source(city)
    else:
        raise ValueError(f"Unsupported weather data source: {source}")

    # Save to cache
    weather_cache[key] = (now, data)
    return data

def fetch_openweathermap(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def fetch_mock_source(city):
    # Mock alternative source - returns dummy data in same structure as OpenWeatherMap
    # This is for demo only - replace with real alternative API calls
    from random import uniform, randint
    now = datetime.utcnow()
    list_data = []
    for i in range(40):  # 5 days * 8 entries per day
        dt = now + timedelta(hours=3*i)
        dt_txt = dt.strftime("%Y-%m-%d %H:%M:%S")
        temp = round(uniform(10, 25), 1)
        rain = round(uniform(0, 2), 1) if randint(0, 1) else 0
        weather_desc = 'clear sky' if randint(0, 2) == 0 else 'rain'
        entry = {
            'dt_txt': dt_txt,
            'main': {'temp': temp},
            'rain': {'3h': rain} if rain > 0 else {},
            'weather': [{'description': weather_desc}]
        }
        list_data.append(entry)

    return {'list': list_data}

## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(data, city):
    temps = [entry['main']['temp'] for entry in data['list']]
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]

    plt.figure(figsize=(12,6))
    plt.plot(times, temps, marker='o', linestyle='-', color='#FF6F61', linewidth=2, label='Temperature (°C)')

    for x, y in zip(times, temps):
        plt.annotate(f"{y:.1f}°C", (x, y), textcoords="offset points", xytext=(0,8), ha='center', fontsize=8)

    plt.title(f"24-Hour Temperature Forecast for {city.title()}", fontsize=16)
    plt.xlabel("Date & Time", fontsize=12)
    plt.ylabel("Temperature (°C)", fontsize=12)
    plt.xticks(rotation=45, fontsize=9)
    plt.yticks(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.legend()
    plt.show()




In [ ]:
def create_precipitation_visualisation(data, city):
    rain_chances = []
    times = []

    for entry in data['list']:
        rain = entry.get('rain', {}).get('3h', 0)
        rain_chances.append(rain)
        times.append(datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S"))

    plt.figure(figsize=(12,6))
    bars = plt.bar(times, rain_chances, width=0.1, color='#69B3E7', edgecolor='black')

    for bar, value in zip(bars, rain_chances):
        if value > 0:
            plt.text(bar.get_x() + bar.get_width()/2, value + 0.1, f"{value:.1f}mm",
                     ha='center', fontsize=8)

    plt.title(f"Precipitation Forecast for {city.title()} (Next 5 Days)", fontsize=16)
    plt.xlabel("Date & Time", fontsize=12)
    plt.ylabel("Rain (mm in 3h)", fontsize=12)
    plt.xticks(rotation=45, fontsize=9)
    plt.yticks(fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()


## 🤖 Natural Language Processing

In [ ]:
def parse_weather_question(question):
    """
    Parses a wider range of natural weather questions.
    Examples supported:
    - "Hows the weather in Paris tomorrow?"
    - "Is it going to rain in London today?"
    - "Will it snow in Berlin?"
    - "What's the weather like in New York?"
    """
    question = question.lower()

    # Try to find condition keywords (rain, snow, clear, cloudy)
    condition_match = re.search(r"(rain|snow|clear|cloudy)", question)

    # Try to find city name by looking for "in "
    city_match = re.search(r"in ([a-zA-Z\s]+?)(?:\s|$|\?)", question)

    # Try to find day (today or tomorrow)
    day_match = re.search(r"(today|tomorrow)", question)

    condition = condition_match.group(1) if condition_match else None
    city = city_match.group(1).strip() if city_match else None
    day = day_match.group(1) if day_match else 'today'

    if city:
        return {'condition': condition, 'city': city, 'day': day}
    else:
        return None


def generate_weather_response(parsed, source='openweathermap'):
    if not parsed:
        return "⚠️ Sorry, I couldn't understand your question. Try asking like: 'Will it rain in Paris tomorrow?' or 'How's the weather in Paris tomorrow?'"

    condition = parsed['condition']
    city = parsed['city']
    day = parsed['day']

    try:
        data = fetch_weather_data(city, source=source)
        target_date = datetime.utcnow().date()
        if day == 'tomorrow':
            target_date += timedelta(days=1)

        # Filter entries for the target date
        entries_for_day = [entry for entry in data['list']
                           if datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S").date() == target_date]

        if not entries_for_day:
            return f"❌ Sorry, no weather data available for {city.title()} on {day}."

        if condition:
            # Check if condition appears in any forecast entry
            found = any(condition in entry['weather'][0]['description'].lower() for entry in entries_for_day)
            if found:
                return f"✅ Yes, it looks like it will be {condition} in {city.title()} {day}."
            else:
                return f"❌ No, it doesn't look like it will be {condition} in {city.title()} {day}."
        else:
            # No condition requested, provide a general summary
            temps = [entry['main']['temp'] for entry in entries_for_day]
            descriptions = [entry['weather'][0]['description'] for entry in entries_for_day]

            avg_temp = sum(temps) / len(temps)
            # Pick the most common weather description for the day
            from collections import Counter
            most_common_desc = Counter(descriptions).most_common(1)[0][0]

            return (f"🌤️ The average temperature in {city.title()} {day} will be around {avg_temp:.1f}°C "
                    f"with mostly {most_common_desc}.")

    except Exception as e:
        return f"❌ Could not retrieve weather for {city.title()}: {e}"

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here

def interactive_ui():
    city_input = widgets.Text(value='', placeholder='Enter city (e.g. Paris)', description='City:')
    question_input = widgets.Text(value='', placeholder='e.g. Will it rain in Paris tomorrow?', description='Ask:')
    source_dropdown = widgets.Dropdown(options=['openweathermap', 'mocksource'], value='openweathermap', description='Source:')
    output_area = widgets.Output()

    forecast_btn = widgets.Button(description="📊 Show Forecast Charts", button_style='info')
    question_btn = widgets.Button(description="🧠 Answer Question", button_style='success')

    def on_forecast_button_click(_):
        with output_area:
            clear_output()
            try:
                data = fetch_weather_data(city_input.value, source=source_dropdown.value)
                create_temperature_visualisation(data, city_input.value)
                create_precipitation_visualisation(data, city_input.value)
            except Exception as e:
                print(f"❌ Error: {e}")

    def on_question_button_click(_):
        with output_area:
            clear_output()
            parsed = parse_weather_question(question_input.value)
            response = generate_weather_response(parsed, source=source_dropdown.value)
            print(response)

    forecast_btn.on_click(on_forecast_button_click)
    question_btn.on_click(on_question_button_click)

    question_section = widgets.VBox([question_input, question_btn], layout=widgets.Layout(margin='10px 0'))
    forecast_section = widgets.VBox([city_input, source_dropdown, forecast_btn], layout=widgets.Layout(margin='10px 0'))

    main_box = widgets.VBox([
        widgets.HTML("🌤️ WeatherWise"),
        question_section,
        widgets.HTML(""),
        forecast_section,
        output_area
    ], layout=widgets.Layout(
        border='1px solid #ddd',
        padding='25px',
        width='500px',
        margin='auto',
        border_radius='12px',
        box_shadow='0 4px 15px rgba(46, 139, 87, 0.3)',
        background_color='#f9fdf8',
        display='flex',
        flex_flow='column nowrap',
        align_items='center'
    ))

    # Wrap in a full-screen flex container to center vertically & horizontally
    wrapper = widgets.Box([main_box], layout=widgets.Layout(
        display='flex',
        align_items='center',
        justify_content='center',
        height='100vh',
        width='100vw',
        background_color='#e6f2e6'
    ))

    display(wrapper)

interactive_ui()

## 🧩 Main Application Logic

In [ ]:
# Tie everything together here

import requests
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import re
from datetime import datetime, timedelta

API_KEY = '7cf20335110caaf78db0fecb31852d45'

# Simple in-memory cache: { (source, city): (timestamp, data) }
weather_cache = {}

CACHE_EXPIRY_MINUTES = 10  # Cache duration

#-------------------------------------------------------------------------------------------------------
# 🌤️ Weather Data Functions
#-------------------------------------------------------------------------------------------------------
# Define get_weather_data() function here
def fetch_weather_data(city, source='openweathermap'):
    """
    Fetch weather data from specified source with caching.
    """
    global weather_cache
    key = (source.lower(), city.lower())

    now = datetime.utcnow()
    if key in weather_cache:
        timestamp, cached_data = weather_cache[key]
        if now - timestamp < timedelta(minutes=CACHE_EXPIRY_MINUTES):

            return cached_data


    if source.lower() == 'openweathermap':
        data = fetch_openweathermap(city)
    elif source.lower() == 'mocksource':
        data = fetch_mock_source(city)
    else:
        raise ValueError(f"Unsupported weather data source: {source}")
    weather_cache[key] = (now, data)
    return data

def fetch_openweathermap(city):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={API_KEY}&units=metric"
    response = requests.get(url)
    response.raise_for_status()
    return response.json()

def fetch_mock_source(city):
    from random import uniform, randint
    now = datetime.utcnow()
    list_data = []
    for i in range(40):
        dt = now + timedelta(hours=3*i)
        dt_txt = dt.strftime("%Y-%m-%d %H:%M:%S")
        temp = round(uniform(10, 25), 1)
        rain = round(uniform(0, 2), 1) if randint(0, 1) else 0
        weather_desc = 'clear sky' if randint(0, 2) == 0 else 'rain'
        entry = {
            'dt_txt': dt_txt,
            'main': {'temp': temp},
            'rain': {'3h': rain} if rain > 0 else {},
            'weather': [{'description': weather_desc}]
        }
        list_data.append(entry)

    return {'list': list_data}

#-------------------------------------------------------------------------------------------------------
# 2. 📊 Visualisation Functions
#-------------------------------------------------------------------------------------------------------
#def create_temperature_visualisation

def create_temperature_visualisation(data, city):
    temps = [entry['main']['temp'] for entry in data['list']]
    times = [datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S") for entry in data['list']]

    plt.figure(figsize=(12,6))
    plt.plot(times, temps, marker='o', linestyle='-', color='#FF6F61', linewidth=2, label='Temperature (°C)')

    for x, y in zip(times, temps):
        plt.annotate(f"{y:.1f}°C", (x, y), textcoords="offset points", xytext=(0,8), ha='center', fontsize=8)

    plt.title(f"24-Hour Temperature Forecast for {city.title()}", fontsize=16)
    plt.xlabel("Date & Time", fontsize=12)
    plt.ylabel("Temperature (°C)", fontsize=12)
    plt.xticks(rotation=45, fontsize=9)
    plt.yticks(fontsize=10)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.legend()
    plt.show()

#create_precipitation_visualisation

def create_precipitation_visualisation(data, city):
    rain_chances = []
    times = []

    for entry in data['list']:
        rain = entry.get('rain', {}).get('3h', 0)
        rain_chances.append(rain)
        times.append(datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S"))

    plt.figure(figsize=(12,6))
    bars = plt.bar(times, rain_chances, width=0.1, color='#69B3E7', edgecolor='black')

    for bar, value in zip(bars, rain_chances):
        if value > 0:
            plt.text(bar.get_x() + bar.get_width()/2, value + 0.1, f"{value:.1f}mm",
                     ha='center', fontsize=8)

    plt.title(f"Precipitation Forecast for {city.title()} (Next 5 Days)", fontsize=16)
    plt.xlabel("Date & Time", fontsize=12)
    plt.ylabel("Rain (mm in 3h)", fontsize=12)
    plt.xticks(rotation=45, fontsize=9)
    plt.yticks(fontsize=10)
    plt.grid(axis='y', linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()


#-------------------------------------------------------------------------------------------------------
# 3. 🤖 Natural Language Processing
#-------------------------------------------------------------------------------------------------------


def parse_weather_question(question):
    """
    Parses a wider range of natural weather questions.
    Examples supported:
    - "Hows the weather in Paris tomorrow?"
    - "Is it going to rain in London today?"
    - "Will it snow in Berlin?"
    - "What's the weather like in New York?"
    """
    question = question.lower()

    condition_match = re.search(r"(rain|snow|clear|cloudy)", question)

    city_match = re.search(r"in ([a-zA-Z\s]+?)(?:\s|$|\?)", question)

    day_match = re.search(r"(today|tomorrow)", question)

    condition = condition_match.group(1) if condition_match else None
    city = city_match.group(1).strip() if city_match else None
    day = day_match.group(1) if day_match else 'today'

    if city:
        return {'condition': condition, 'city': city, 'day': day}
    else:
        return None


def generate_weather_response(parsed, source='openweathermap'):
    if not parsed:
        return "⚠️ Sorry, I couldn't understand your question. Try asking like: 'Will it rain in Paris tomorrow?' or 'How's the weather in Paris tomorrow?'"

    condition = parsed['condition']
    city = parsed['city']
    day = parsed['day']

    try:
        data = fetch_weather_data(city, source=source)
        target_date = datetime.utcnow().date()
        if day == 'tomorrow':
            target_date += timedelta(days=1)

        entries_for_day = [entry for entry in data['list']
                           if datetime.strptime(entry['dt_txt'], "%Y-%m-%d %H:%M:%S").date() == target_date]

        if not entries_for_day:
            return f"❌ Sorry, no weather data available for {city.title()} on {day}."

        if condition:

            found = any(condition in entry['weather'][0]['description'].lower() for entry in entries_for_day)
            if found:
                return f"✅ Yes, it looks like it will be {condition} in {city.title()} {day}."
            else:
                return f"❌ No, it doesn't look like it will be {condition} in {city.title()} {day}."
        else:

            temps = [entry['main']['temp'] for entry in entries_for_day]
            descriptions = [entry['weather'][0]['description'] for entry in entries_for_day]

            avg_temp = sum(temps) / len(temps)
            from collections import Counter
            most_common_desc = Counter(descriptions).most_common(1)[0][0]

            return (f"🌤️ The average temperature in {city.title()} {day} will be around {avg_temp:.1f}°C "
                    f"with mostly {most_common_desc}.")

    except Exception as e:
        return f"❌ Could not retrieve weather for {city.title()}: {e}"

#-------------------------------------------------------------------------------------------------------
# 4. 🧭 User Interface
#-------------------------------------------------------------------------------------------------------

def interactive_ui():
    city_input = widgets.Text(value='', placeholder='Enter city (e.g. Paris)', description='City:')
    question_input = widgets.Text(value='', placeholder='e.g. Will it rain in Paris tomorrow?', description='Ask:')
    source_dropdown = widgets.Dropdown(options=['openweathermap', 'mocksource'], value='openweathermap', description='Source:')
    output_area = widgets.Output()

    forecast_btn = widgets.Button(description="📊 Show Forecast Charts", button_style='info')
    question_btn = widgets.Button(description="🧠 Answer Question", button_style='success')

    def on_forecast_button_click(_):
        with output_area:
            clear_output()
            try:
                data = fetch_weather_data(city_input.value, source=source_dropdown.value)
                create_temperature_visualisation(data, city_input.value)
                create_precipitation_visualisation(data, city_input.value)
            except Exception as e:
                print(f"❌ Error: {e}")

    def on_question_button_click(_):
        with output_area:
            clear_output()
            parsed = parse_weather_question(question_input.value)
            response = generate_weather_response(parsed, source=source_dropdown.value)
            print(response)

    forecast_btn.on_click(on_forecast_button_click)
    question_btn.on_click(on_question_button_click)

    question_section = widgets.VBox([question_input, question_btn], layout=widgets.Layout(margin='10px 0'))
    forecast_section = widgets.VBox([city_input, source_dropdown, forecast_btn], layout=widgets.Layout(margin='10px 0'))

    main_box = widgets.VBox([
        widgets.HTML("🌤️ WeatherWise"),
        question_section,
        widgets.HTML(""),
        forecast_section,
        output_area
    ], layout=widgets.Layout(
        border='1px solid #ddd',
        padding='25px',
        width='500px',
        margin='auto',
        border_radius='12px',
        box_shadow='0 4px 15px rgba(46, 139, 87, 0.3)',
        background_color='#f9fdf8',
        display='flex',
        flex_flow='column nowrap',
        align_items='center'
    ))

    # Wrap in a full-screen flex container to center vertically & horizontally
    wrapper = widgets.Box([main_box], layout=widgets.Layout(
        display='flex',
        align_items='center',
        justify_content='center',
        height='100vh',
        width='100vw',
        background_color='#e6f2e6'
    ))

    display(wrapper)

interactive_ui()

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

#-------------------------------------------------------------------------------------------------------
# 1.🌤️ Weather Data Functions
#-------------------------------------------------------------------------------------------------------

# 1.1 ✅ Define the get_weather_data() Function
from typing import Union, List, Dict

def get_weather_data(location: Union[str, List[str]], forecast_days: int = 5) -> Dict[str, dict]:
    """
    Fetch weather data for one or more locations and return it as a dictionary.

    Args:
        location (str or list): A single location name or a list of locations.
        forecast_days (int): Number of forecast days to retrieve (default is 5).

    Returns:
        dict: A dictionary mapping each location to its weather data or error.
    """
    if isinstance(location, str):
        location = [location]

    results = {}

    for loc in location:
        try:
            weather_response = get_weather(loc)
            if not hasattr(weather_response, 'dict'):
                raise TypeError("Weather object does not support dict() conversion.")

            weather_data = weather_response.dict()
            forecast = weather_data.get("forecast", [])
            current = weather_data.get("current_conditions", {})

            if not forecast:
                raise ValueError("Forecast data is missing or empty.")

            results[loc] = {
                "current_conditions": current or "No current data available",
                "forecast": forecast[:forecast_days]
            }

        except Exception as e:
            results[loc] = {"error": f"Failed to retrieve weather for {loc}: {e}"}

    return results

# 1.2 ✅ Run doctest

import doctest
doctest.testmod()


def get_weather_data(location: Union[str, List[str]], forecast_days: int = 5) -> Dict[str, dict]:
    """
    >>> isinstance(get_weather_data("Paris"), dict)
    True
    >>> "error" in get_weather_data("FakeCity")["FakeCity"]
    True
    """

# 1.3 ✅ Run unittest

import unittest

class MockWeatherResponse:
    def __init__(self, valid=True):
        self.valid = valid

    def dict(self):
        if self.valid:
            return {
                "current_conditions": {"temperature": 20},
                "forecast": [{"day": 1}, {"day": 2}, {"day": 3}]
            }
        else:
            return {}

def get_weather(location):
    if location == "FakeCity":
        raise ValueError("Location not found")
    return MockWeatherResponse()

class TestGetWeatherData(unittest.TestCase):
    def test_valid_city(self):
        result = get_weather_data("Paris", forecast_days=2)
        self.assertIn("current_conditions", result["Paris"])
        self.assertEqual(len(result["Paris"]["forecast"]), 2)

    def test_invalid_city(self):
        result = get_weather_data("FakeCity")
        self.assertIn("error", result["FakeCity"])

unittest.main(argv=[''], verbosity=2, exit=False)

# 1.4 🐞 Debugging

def debug_locations(locations):
    print("🌍 Fetching weather for multiple locations...")
    results = get_weather_data(locations)
    for loc, data in results.items():
        print(f"\n📍 Location: {loc}")
        if "error" in data:
            print(f"❌ Error: {data['error']}")
        else:
            print(f"✅ Current: {data['current_conditions']}")
            print(f"📅 Forecast ({len(data['forecast'])} days):")
            for day in data['forecast']:
                print(day)

debug_locations(["London", "FakeCity", "Berlin"])

#-------------------------------------------------------------------------------------------------------
# 2. 📊 Visualisation Functions
#-------------------------------------------------------------------------------------------------------

# 2.1 Define the Visualisation Functions

# 2.1.1 Temperature Line Chart (Plotly)

import plotly.express as px
import pandas as pd

def create_temperature_visualisation_interactive(weather_data):
    """
    Create an interactive temperature line chart using Plotly.
    """
    rows = []
    for location, times_data in weather_data.items():
        for time, info in times_data.items():
            rows.append({
                "Location": location,
                "Time": time,
                "Temperature": info["temperature"]
            })

    df = pd.DataFrame(rows)
    fig = px.line(
        df, x="Time", y="Temperature", color="Location",
        markers=True, title="Interactive Temperature Forecast",
        labels={"Temperature": "Temp (°C)", "Time": "Time Period"}
    )
    fig.update_traces(mode='lines+markers', hovertemplate="%{y}°C at %{x}<br>%{fullData.name}")
    fig.update_layout(xaxis_tickangle=-45)
    fig.show()

# 2.1.2 Precipitation Bar Chart (Matplotlib)

import pandas as pd
import matplotlib.pyplot as plt

def create_precipitation_visualisation_from_df(df: pd.DataFrame, output_type='display'):
    """
    Create a grouped bar chart of precipitation chances using a pandas DataFrame.
    """
    plt.style.use('seaborn-darkgrid')
    fig, ax = plt.subplots(figsize=(10, 6))

    locations = df['Location'].unique()
    time_labels = df['Time'].unique()
    width = 0.8 / len(locations)
    x = range(len(time_labels))

    for idx, loc in enumerate(locations):
        loc_data = df[df['Location'] == loc]
        loc_data = loc_data.set_index('Time').reindex(time_labels).reset_index()
        offsets = [i + (idx * width) for i in x]
        precip = loc_data['Precipitation'].fillna(0).tolist()

        ax.bar(offsets, precip, width=width, label=loc)
        for i, value in enumerate(precip):
            ax.text(offsets[i], value + 1, f"{value}%", ha='center', fontsize=8)

    ax.set(
        title="Chance of Rain by Location",
        xlabel="Time Period",
        ylabel="Chance of Rain (%)"
    )
    ax.set_xticks([i + width * (len(locations)-1)/2 for i in x])
    ax.set_xticklabels(time_labels, rotation=45)
    ax.legend()
    ax.grid(True)

    if output_type == 'figure':
        return fig
    plt.show()

# 2.2 ✅ Run doctest for Format Check

import doctest
doctest.testmod()


def create_temperature_visualisation_interactive(weather_data):
    """
    >>> isinstance(weather_data, dict)
    True
    """
    ...

# 2.3 ✅ Run unittest for Chart Functions

import unittest

class TestVisualisationData(unittest.TestCase):
    def test_dataframe_conversion(self):
        data = {
            "London": {"Mon": {"temperature": 21}, "Tue": {"temperature": 22}},
            "Paris": {"Mon": {"temperature": 19}, "Tue": {"temperature": 20}},
        }
        rows = []
        for location, times_data in data.items():
            for time, info in times_data.items():
                rows.append({
                    "Location": location,
                    "Time": time,
                    "Temperature": info["temperature"]
                })
        df = pd.DataFrame(rows)
        self.assertEqual(len(df), 4)
        self.assertIn("Location", df.columns)

unittest.main(argv=[''], verbosity=2, exit=False)

# 2.4 ✅ Run assert Tests (Pytest Style)

sample_data = {
    "London": {"Monday": {"temperature": 20}, "Tuesday": {"temperature": 22}},
    "Paris": {"Monday": {"temperature": 18}, "Tuesday": {"temperature": 21}}
}

def test_temperature_data():
    rows = []
    for location, times_data in sample_data.items():
        for time, info in times_data.items():
            rows.append({
                "Location": location,
                "Time": time,
                "Temperature": info["temperature"]
            })
    df = pd.DataFrame(rows)
    assert df.shape[0] == 4
    assert "Temperature" in df.columns

test_temperature_data()
print("All visualisation data tests passed.")

# 2.5 🐞 Debug with Sample Charts

# Temperature (Interactive)
create_temperature_visualisation_interactive({
    "London": {"Monday": {"temperature": 20}, "Tuesday": {"temperature": 22}},
    "Paris": {"Monday": {"temperature": 18}, "Tuesday": {"temperature": 21}}
})

# Precipitation (Grouped Bar)
df = pd.DataFrame({
    "Location": ["London", "Paris", "London", "Paris"],
    "Time": ["Monday", "Monday", "Tuesday", "Tuesday"],
    "Precipitation": [30, 50, 10, 60]
})
create_precipitation_visualisation_from_df(df)

#-------------------------------------------------------------------------------------------------------
# 3. 🤖 Natural Language Processing
#-------------------------------------------------------------------------------------------------------

# 3.1 ✅ Define generate_weather_response()

def generate_weather_response(parsed, data, temp_unit='Celsius', wind_unit='km/h'):
    loc = parsed['location']
    time = parsed['time_period']
    attr = parsed['attribute']

    if loc not in data or time not in data[loc]:
        return f"❌ No forecast available for {loc} {time}."

    forecast = data[loc][time]

    def convert_temp(c): return f"{convert_temperature(c, temp_unit)}°{'F' if temp_unit == 'Fahrenheit' else 'C'}"
    def convert_wind(w): return f"{convert_wind_speed(w, wind_unit)} {wind_unit}"

    responses = {
        'temperature': lambda: f"🌡️ Temperature in {loc} {time} is {convert_temp(forecast.get('temperature', 'unknown'))}.",
        'rain': lambda: f"🌧️ Expect {'rain' if forecast.get('rain') else 'no rain'} in {loc} {time}.",
        'snow': lambda: f"❄️ Expect {'snow' if forecast.get('snow') else 'no snow'} in {loc} {time}.",
        'wind': lambda: f"💨 Wind speed in {loc} {time} is {convert_wind(forecast.get('wind', 'unknown'))}.",
        'humidity': lambda: f"💧 Humidity in {loc} {time} is {forecast.get('humidity', 'unknown')}%."
    }

    if attr in responses:
        return responses[attr]()
    else:
        temp = convert_temp(forecast.get('temperature', 'unknown'))
        rain = 'rain' if forecast.get('rain') else 'no rain'
        return f"In {loc} {time}, temperature is {temp} with {rain}."

# 3.2 ✅ Add doctest

def generate_weather_response(parsed, data, temp_unit='Celsius', wind_unit='km/h'):
    """
    >>> parsed = {'location': 'Paris', 'time_period': 'today', 'attribute': 'temperature'}
    >>> data = {'Paris': {'today': {'temperature': 20}}}
    >>> generate_weather_response(parsed, data)
    '🌡️ Temperature in Paris today is 20°C.'
    """

import doctest
doctest.testmod()

# 3.3 ✅ Unit Testing with unittest

# Mock functions for temperature/wind conversion:

def convert_temperature(value, unit):
    return round((value * 9/5 + 32) if unit == 'Fahrenheit' else value)

def convert_wind_speed(value, unit):
    return round((value / 1.609) if unit == 'mph' else value)

# Then test:
import unittest

class TestGenerateWeatherResponse(unittest.TestCase):
    def test_temperature_celsius(self):
        parsed = {'location': 'Paris', 'time_period': 'today', 'attribute': 'temperature'}
        data = {'Paris': {'today': {'temperature': 25}}}
        result = generate_weather_response(parsed, data)
        self.assertIn("25°C", result)

    def test_missing_location(self):
        parsed = {'location': 'Mars', 'time_period': 'today', 'attribute': 'temperature'}
        data = {'Paris': {'today': {'temperature': 25}}}
        result = generate_weather_response(parsed, data)
        self.assertIn("❌", result)

unittest.main(argv=[''], verbosity=2, exit=False)

# 3.4 ✅ Use assert (Pytest-style tests)

parsed = {'location': 'Paris', 'time_period': 'today', 'attribute': 'wind'}
data = {'Paris': {'today': {'wind': 20}}}

def test_weather_response():
    result = generate_weather_response(parsed, data, wind_unit='km/h')
    assert "20 km/h" in result
    assert "💨" in result

test_weather_response()
print("All pytest-style tests passed.")

# 3.5 🐞 Debug with Sample Data

parsed_inputs = [
    {'location': 'London', 'time_period': 'today', 'attribute': 'temperature'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'rain'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'wind'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'snow'},
    {'location': 'London', 'time_period': 'today', 'attribute': 'general'},
]

weather_data = {
    "London": {
        "today": {
            "temperature": 22,
            "rain": False,
            "snow": False,
            "wind": 15,
            "humidity": 70
        }
    }
}

for parsed in parsed_inputs:
    print(generate_weather_response(parsed, weather_data))

#-------------------------------------------------------------------------------------------------------
# 4. 🧭 User Interface
#-------------------------------------------------------------------------------------------------------
# 4.1 ✅ Define the Menu Interface Functions

import pyinputplus as pyip

def print_banner():
    print("\n" + "="*40)
    print("☁️  Welcome to the Weather Assistant!  ☁️")
    print("="*40)

def main_menu():
    print("\n=== 🌦️ Weather Assistant Menu ===")
    return pyip.inputMenu(
        ["🌤️  Ask Weather Question", "📊  Visualize Weather Data", "🚪  Exit"],
        lettered=True,
        numbered=False,
        prompt="\n🌈 What would you like to do?\n",
        blank=False,
        limit=3,
        retries=2,
        default="🚪  Exit"
    )

def visualization_menu():
    return pyip.inputMenu(
        ["📈  Temperature", "🌧️  Precipitation", "🔙  Back to Main Menu"],
        lettered=True,
        numbered=False,
        prompt="\n📊 Choose a visualization type:\n",
        blank=False,
        limit=3,
        retries=2,
        default="🔙  Back to Main Menu"
    )

def run_interface():
    print_banner()
    while True:
        choice = main_menu()
        if "Ask Weather" in choice:
            print("🗣️  You chose to ask a weather question.")
        elif "Visualize" in choice:
            while True:
                sub_choice = visualization_menu()
                if "Temperature" in sub_choice:
                    print("📈 Showing temperature chart...")
                elif "Precipitation" in sub_choice:
                    print("🌧️ Showing precipitation chart...")
                elif "Back" in sub_choice:
                    break
        elif "Exit" in choice:
            print("👋 Exiting Weather Assistant. Stay safe!")
            break

# 4.2 ✅ Add Doctests

def main_menu():
    """
    >>> isinstance(main_menu(), str)
    True
    """

import doctest
doctest.testmod()

# 4.3 ✅ Unittest for Menu Logic

import unittest

class TestMenuStructure(unittest.TestCase):
    def test_main_menu_options(self):
        options = [
            "🌤️  Ask Weather Question",
            "📊  Visualize Weather Data",
            "🚪  Exit"
        ]
        self.assertEqual(len(options), 3)
        self.assertIn("🌤️  Ask Weather Question", options)

unittest.main(argv=[''], verbosity=2, exit=False)


# 4.4 ✅ Assert-Based Tests (Pytest-style)

def test_menu_choice_return_type():
    menu_response = "🌧️  Precipitation"
    assert isinstance(menu_response, str)
    assert "Precipitation" in menu_response

test_menu_choice_return_type()
print("✅ Menu tests passed.")

# 4.5 🐞 Debug with Manual Simulation

# Run this in a local terminal or Colab (with ipywidgets disabled):
run_interface()

# Expected steps:
# - Shows banner
# - Main menu with 3 choices
# - If 'Visualize Weather Data' is chosen:
#     - Shows 3 submenu options
#     - Returns to main menu on 'Back'


